In [1]:
# !pip install -q transformers datasets accelerate

In [2]:
!pip install evaluate
!pip install faiss-gpu-cu12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api")

In [4]:
import wandb
wandb.login(key=secret_value_0)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: iakarshu to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
from datasets import load_dataset, DatasetDict
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer)
import torch, json, numpy as np
from sentence_transformers import SentenceTransformer
import plotly.express as px

2025-06-06 20:00:10.612184: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749240010.798778      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749240010.852425      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
import pandas as pd
from tqdm.auto import tqdm
pd.set_option('max_colwidth', None)

In [7]:
# Parameters

model_name = "microsoft/deberta-v3-base"

In [8]:
# 1. Load SST-2
ds = load_dataset("nyu-mll/glue", "sst2")          
tok   = AutoTokenizer.from_pretrained(model_name)

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

...

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


## Understanding some stats around the training / val data

In [9]:
fig = px.histogram(x = ds['train']['label'], color = ds['train']['label'])
fig.show()

In [10]:
## Sentence length distribution

df = ds["train"].to_pandas()            # columns: sentence, label

# ------------------------------------------------------------------
# 2. Add a sentence-length column  (word count)
# ------------------------------------------------------------------
df["sent_len"] = df["sentence"].str.split().apply(len)

# Optional: map 0/1 to readable names
label_map = {0: "negative", 1: "positive"}
df["label_name"] = df["label"].map(label_map)

# ------------------------------------------------------------------
# 3. Plotly histogram with marginal box-plots
# ------------------------------------------------------------------
fig = px.histogram(
    df,
    x="sent_len",
    color="label_name",
    nbins=50,
    barmode="overlay",        # overlay bars; use 'group' if you prefer side-by-side
    opacity=0.6,
    marginal="box",           # adds a little box-plot above each distribution
    title="Sentence-length distribution in SST-2 train split",
    labels={"sent_len": "Sentence length (words)", "label_name": "Sentiment"}
)

fig.update_layout(legend_title_text="Sentiment class")
fig.show()

In [11]:
fig = px.histogram(x = ds['validation']['label'], color = ds['validation']['label'])
fig.show()

In [12]:
## Sentence length distribution

df = ds["validation"].to_pandas()            # columns: sentence, label

# ------------------------------------------------------------------
# 2. Add a sentence-length column  (word count)
# ------------------------------------------------------------------
df["sent_len"] = df["sentence"].str.split().apply(len)

# Optional: map 0/1 to readable names
label_map = {0: "negative", 1: "positive"}
df["label_name"] = df["label"].map(label_map)

# ------------------------------------------------------------------
# 3. Plotly histogram with marginal box-plots
# ------------------------------------------------------------------
fig = px.histogram(
    df,
    x="sent_len",
    color="label_name",
    nbins=50,
    barmode="overlay",        # overlay bars; use 'group' if you prefer side-by-side
    opacity=0.6,
    marginal="box",           # adds a little box-plot above each distribution
    title="Sentence-length distribution in SST-2 validation split",
    labels={"sent_len": "Sentence length (words)", "label_name": "Sentiment"}
)

fig.update_layout(legend_title_text="Sentiment class")
fig.show()

## Performing the training related things

In [13]:
def tokenize(ex): 
    return tok(ex["sentence"], truncation=True)
ds_tok = ds.map(tokenize, batched=True).rename_column("label", "labels")

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [14]:
ds_tok

DatasetDict({
    train: Dataset({
        features: ['sentence', 'labels', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'labels', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'labels', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [15]:
# 2. Model -------------------------------------------------------------------
model = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels=2,
            # drop FP32 embeddings to save VRAM
            #torch_dtype="bfloat16"
)

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# 3. Training setup ----------------------------------------------------------
args = TrainingArguments(
    "sst2-deberta-v3",
    fp16=True,                             # mixed precision
    per_device_train_batch_size=48,         # fits on T4 16 GB
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    learning_rate=2e-5,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [17]:
trainer = Trainer(model, args,
                  train_dataset=ds_tok["train"],
                  eval_dataset=ds_tok["validation"],
                  tokenizer=tok)

/tmp/ipykernel_19/1501121623.py:1: FutureWarning:

`tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.



In [18]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Tracking run with wandb version 0.19.9
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250606_200040-b9iw5563
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run sst2-deberta-v3
wandb: ⭐️ View project at https://wandb.ai/iakarshu/huggingface
wandb: 🚀 View run at https://wandb.ai/iakarshu/huggingface/runs/b9iw5563


Epoch,Training Loss,Validation Loss
1,0.151600,0.148793
2,0.094400,0.146381


TrainOutput(global_step=2808, training_loss=0.13762517600317625, metrics={'train_runtime': 792.4812, 'train_samples_per_second': 169.97, 'train_steps_per_second': 3.543, 'total_flos': 2812620260887560.0, 'train_loss': 0.13762517600317625, 'epoch': 2.0})

## Performing the error analysis (loading the best checkpoint)

In [19]:
model_name = "/kaggle/working/sst2-deberta-v3/checkpoint-1404" ## It is microsoft/deberta-base-v3

In [20]:
# 2. Model -------------------------------------------------------------------
model = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            # drop FP32 embeddings to save VRAM
            #torch_dtype="bfloat16"
)

# 3. Training setup ----------------------------------------------------------
args = TrainingArguments(
    "sst2-deberta-v3",
    fp16=True,                             # mixed precision
    per_device_train_batch_size=48,         # fits on T4 16 GB
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    learning_rate=2e-5,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(model, args,
                  train_dataset=ds_tok["train"],
                  eval_dataset=ds_tok["validation"],
                  tokenizer=tok)


# 4. Collect logits for error analysis --------------------------------------
pred_train = trainer.predict(ds_tok["train"])
pred_dev   = trainer.predict(ds_tok["validation"])

/tmp/ipykernel_19/279399822.py:21: FutureWarning:

`tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.



In [21]:
import numpy as np, pandas as pd
from scipy.special import softmax

def collect_errors(pred_out, split_name, raw_ds):
    logits   = pred_out.predictions
    gold     = pred_out.label_ids
    preds    = logits.argmax(axis=1)
    probs    = softmax(logits, axis=1)
    sent_col = raw_ds[split_name]["sentence"]

    wrong_mask = preds != gold
    idx        = np.where(wrong_mask)[0]

    return pd.DataFrame({
        "idx"       : idx,
        "split"     : split_name,
        "sentence"  : [sent_col[i]        for i in idx],
        "gold"      : gold[idx],
        "pred"      : preds[idx],
        "p_pred"    : probs[idx, preds[idx]],
        "p_gold"    : probs[idx, gold[idx]],
    })

# Combine train + dev errors
errors_df = pd.concat([
    collect_errors(pred_train, "train", ds),
    collect_errors(pred_dev,   "validation", ds)
]).reset_index(drop=True)

print(errors_df.groupby("split").size())

errors_df.to_csv("sst2_deberta_errors.csv", index=False)

split
train         1936
validation      38
dtype: int64


In [22]:
errors_df = pd.read_csv("sst2_deberta_errors.csv")

In [23]:
from evaluate import load
import numpy as np

metric = load("glue", "sst2")          # loads the official scorer

dev_acc = 100.0 * metric.compute(
    predictions = pred_dev.predictions.argmax(axis=1),
    references   = pred_dev.label_ids
)["accuracy"]

train_acc = 100.0 * metric.compute(
    predictions = pred_train.predictions.argmax(axis=1),
    references   = pred_train.label_ids
)["accuracy"]

print(f"DeBERTa-v3-base  •  train acc: {train_acc:.4f}  •  dev acc: {dev_acc:.4f}")

DeBERTa-v3-base  •  train acc: 97.1254  •  dev acc: 95.6422


In [24]:
train_set_size = len(ds_tok['train'])
val_set_size = len(ds_tok['validation'])

In [25]:
train_error = errors_df[errors_df['split']=='train'].shape[0]
val_error = errors_df[errors_df['split']=='validation'].shape[0]

percentage_error_train = 100.0 * train_error / train_set_size
percentage_error_val = 100.0 * val_error / val_set_size

print(f"DeBERTa-v3-base  •  % of train incorrect: {percentage_error_train:.4f} {train_error}  •  % of dev incorrect: {percentage_error_val:.4f} {val_error}")

DeBERTa-v3-base  •  % of train incorrect: 2.8746 1936  •  % of dev incorrect: 4.3578 38


## 1 Create embeddings & cluster the mis-predictions

In [26]:
err_train = errors_df[errors_df.split == "train"].reset_index(drop=True)
err_dev   = errors_df[errors_df.split == "validation"].reset_index(drop=True)

In [27]:
err_train.sort_values(by = 'p_pred', ascending = False)[['sentence', "gold", "pred", 'p_pred']]

,sentence,gold,pred,p_pred
1214,a damn fine and a truly distinctive and a deeply pertinent film,0,1,0.999562
699,provides an invaluable service,0,1,0.999541
906,of great actors hamming it up,0,1,0.999505
1095,"is , truly and thankfully , a one-of-a-kind work",0,1,0.999482
1791,as original and insightful as last week 's episode of behind the music .,0,1,0.999440
...,...,...,...,...
729,"a movie with a ` children 's ' song that includes the line ` my stepdad 's not mean , he 's just adjusting '",0,1,0.501755
981,"a young woman who wants many things in life , but fears she 'll become her mother before she gets to fulfill her dreams",1,0,0.501648
26,out,0,1,0.501587
31,"( or role , or edit , or score , or anything , really",0,1,0.500305


In [28]:
from sentence_transformers import SentenceTransformer
st = SentenceTransformer("avsolatorio/GIST-large-Embedding-v0",
                         device="cuda")   # "cpu" also works

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/754 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [29]:
err_dev["emb"]   = st.encode(err_dev.sentence.tolist(),
                             convert_to_numpy=True,
                             normalize_embeddings=True).tolist()
err_train["emb"] = st.encode(err_train.sentence.tolist(),
                             convert_to_numpy=True,
                             normalize_embeddings=True).tolist()

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/61 [00:00<?, ?it/s]

In [30]:
from sklearn.cluster import AgglomerativeClustering
import numpy as np

X = np.vstack(err_dev.emb.values)
clust = AgglomerativeClustering(
            n_clusters=None,            # auto decide
            metric="cosine",
            linkage="average",
            distance_threshold=0.7
       ).fit(X)

err_dev["cluster_id"] = clust.labels_

In [31]:
X = np.vstack(err_train.emb.values)
clust = AgglomerativeClustering(
            n_clusters=None,            # auto decide
            metric="cosine",
            linkage="average",
            distance_threshold=0.7
       ).fit(X)

err_train["cluster_id"] = clust.labels_

## Nearest-neighbour lookup in the training set

In [32]:
# ---- 1.1  Prepare plain-text list -----------------------------------------
train_texts  = ds["train"]["sentence"]          # original, un-tokenised text
train_labels = ds["train"]["label"]             # 0 = neg, 1 = pos

# ---- 1.2  Encode in batches (GPU if available) -----------------------------
BATCH = 256
emb_list = []
for i in tqdm(range(0, len(train_texts), BATCH), total = len(train_texts) // BATCH):
    batch = train_texts[i : i+BATCH]
    emb   = st.encode(batch,
                      batch_size=len(batch),
                      convert_to_numpy=True,
                      normalize_embeddings=True,
                      show_progress_bar=False)
    emb_list.append(emb.astype("float32"))      # FAISS needs float32
train_emb = np.vstack(emb_list)                 # shape (67 k, 768)

  0%|          | 0/263 [00:00<?, ?it/s]

In [33]:
import faiss

d = train_emb.shape[1]                # embedding dimensionality = 768
index = faiss.IndexFlatIP(d)          # IP on unit-norm vectors == cosine
index.add(train_emb)                  # adds all 67 k vectors

In [34]:
def fetch_neighbours(query_vecs, k=5, same_split=False):
    """
    query_vecs : np.ndarray  (n_queries, 768)  – unit-norm
    k          : int         – number of neighbours to return
    same_split : bool        – if True, remove self-hits for training errors
    """
    sim, idx = index.search(query_vecs.astype("float32"), k + (1 if same_split else 0))
    # idx : shape (n_queries, k+1)
    neighbours = []
    for row_idx, row in enumerate(idx):
        out = []
        for j in row:
            if j == -1:
                continue
            # optional: skip self-example for train errors
            if same_split and j == err_train.iloc[row_idx]["idx"]:
                continue
            out.append(dict(
                sent = train_texts[j],
                label = int(train_labels[j]),
                sim   = float(sim[row_idx, np.where(row == j)[0][0]])
            ))
            if len(out) == k:
                break
        neighbours.append(out)
    return neighbours


In [35]:
dev_vecs = np.vstack(err_dev.emb.values)
err_dev["train_neighbours"] = fetch_neighbours(dev_vecs, k=5, same_split=False)

In [36]:
train_vecs = np.vstack(err_train.emb.values)
err_train["train_neighbours"] = fetch_neighbours(train_vecs, k=5, same_split=True)

In [37]:
idx = 1

print(err_train.iloc[idx][["sentence", "gold", "p_gold", "pred", "p_pred"]])
for nbr in err_train.iloc[idx]["train_neighbours"]:
    print(f'  · ({nbr["label"]})  sim={nbr["sim"]:.2f}  {nbr["sent"]}')

sentence      a doa 
gold               0
p_gold      0.355104
pred               1
p_pred      0.644896
Name: 1, dtype: object
  · (0)  sim=0.93  is doa . 
  · (0)  sim=0.91  here is doa . 
  · (0)  sim=0.91  is doa 
  · (0)  sim=0.78  a doa dud from frame one . 
  · (0)  sim=0.69  is a dud . 


## Correcting training data

In [38]:
import numpy as np

# --- Config -----------------------------------------------------------------
K_NEIGHBORS      = 5       # how many nearest neighbours to inspect
SIM_THRESHOLD    = 0.80    # only neighbours with cosine-sim ≥ τ are trusted
MAJORITY_REQ     = 0.80    # ≥80 % of those neighbours must agree
MIN_CONTEXT_NEBS = 3       # otherwise we call it "not enough context"


def _majority_label(labels: list[int]) -> tuple[int, float]:
    """
    Return (most frequent label, ratio_of_majority).
    Example: [1,1,1,0,1] -> (1, 0.8)
    """
    vals, counts = np.unique(labels, return_counts=True)
    idx = counts.argmax()
    return int(vals[idx]), counts[idx] / len(labels)


def categorise_error(row):
    """
    Classify a mis-prediction into:
      'no_context'   – < MIN_CONTEXT_NEBS neighbours above SIM_THRESHOLD
      'suspect_mislabel' – neighbours strongly disagree with gold label
      'model_error'  – neighbours back the gold label → true model miss
    Also returns the suggested flip (0/1) for mis‐labels, else None.
    """
    # 1. keep only similar neighbours
    neighbours = [n for n in row.train_neighbours if n["sim"] >= SIM_THRESHOLD]

    if len(neighbours) < MIN_CONTEXT_NEBS:
        return "no_context", None

    labels = [n["label"] for n in neighbours[:K_NEIGHBORS]]
    maj_label, maj_ratio = _majority_label(labels)

    if (maj_label != row.gold) and (maj_ratio >= MAJORITY_REQ):
        return "suspect_mislabel", maj_label

    return "model_error", None

In [39]:
for df in (err_train, err_dev):
    df[["error_type", "suggested_flip"]] = df.apply(
        categorise_error, axis=1, result_type="expand"
    )

In [40]:
import pandas as pd
import plotly.express as px

# merge the two frames for plotting
errs_all = pd.concat([err_train, err_dev], ignore_index=True)

# proportion per split
counts = (errs_all
          .groupby(["split", "error_type"])
          .size()
          .reset_index(name="count"))

counts["percent"] = (
    counts["count"] /
    counts.groupby("split")["count"].transform("sum")   # same length Series
    * 100
)

print(counts.dtypes)


fig = px.bar(
    counts,
    x="error_type",
    y="percent",
    color="error_type",
    facet_col="split",           # separate panels for train / validation
    text=counts["percent"].round(1).astype(str) + "%" + " Absolute value:" + counts['count'].astype(str),
    title="Break-down of DeBERTa-v3-base mis-classifications on SST-2",
    category_orders={"error_type": ["model_error", "suspect_mislabel", "no_context"]}
)
fig.update_yaxes(title="% of mis-classified samples", ticksuffix="%")
fig.update_traces(textposition="outside")

split          object
error_type     object
count           int64
percent       float64
dtype: object


/tmp/ipykernel_19/3746069449.py:5: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [41]:
from pathlib import Path
import pandas as pd
from datasets import Dataset, DatasetDict

# ------------------------------------------------------------------
# 1.  Patch the train split (flip suspect_mislabel rows)
# ------------------------------------------------------------------
train_df = ds["train"].to_pandas()

flip_map = (err_train                      # built earlier
            .query("error_type == 'suspect_mislabel'")
            .set_index("idx")["suggested_flip"]
            .dropna()
            .astype(int))

In [42]:
flip_map

idx
285      0
319      0
645      0
710      1
827      1
        ..
66646    0
66648    1
66691    0
66706    1
67028    1
Name: suggested_flip, Length: 339, dtype: int64

In [43]:
err_train[err_train['sentence'].apply(lambda x : "breezy" in x)]

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning:

invalid value encountered in greater

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning:

invalid value encountered in less

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning:

invalid value encountered in greater



,idx,split,sentence,gold,pred,p_pred,p_gold,emb,cluster_id,train_neighbours,error_type,suggested_flip
6,263,train,"breezy , distracted rhythms",0,1,0.903677,0.096323,"[-0.015471036545932293, 0.03173375129699707, 0.005166224204003811, 0.013572033494710922, -0.02925446443259716, -0.04478578642010689, -0.009715585969388485, -0.03298046812415123, 0.00965580902993679, -0.017652230337262154, 0.006591772194951773, -0.010592130944132805, -0.028563225641846657, -0.03975997865200043, 0.009127888828516006, 0.008252281695604324, -0.08089068531990051, -0.02802884764969349, -0.06102413311600685, 0.02220170572400093, 0.016113409772515297, 0.021726083010435104, -0.01548528578132391, -0.04048274829983711, 0.015235206112265587, 0.02278129570186138, -0.03363616764545441, 0.03332192450761795, 0.024833960458636284, 0.057849761098623276, 0.054089512676000595, -0.007183017209172249, -0.042622774839401245, -0.011664963327348232, 0.0047478643245995045, -0.0021102731116116047, 0.002619026228785515, -0.0045616514980793, 0.019922541454434395, -0.02785991132259369, -0.0042813061736524105, -0.0635470375418663, -0.01629060134291649, -0.013224736787378788, 0.011441417969763279, -0.045401137322187424, -0.010386621579527855, -0.0412895567715168, 0.0792703926563263, -0.038975756615400314, -0.008255008608102798, -0.003632286563515663, -0.01670864224433899, 0.02395048178732395, -0.004590259399265051, 0.02993246354162693, 0.009356867522001266, 0.021289406344294548, 0.026012852787971497, 0.0352017767727375, 0.02702341601252556, 0.05711568146944046, 0.02355930209159851, -0.09422793239355087, -0.03352173790335655, 0.028140053153038025, -0.01429121382534504, -0.04117009788751602, -0.032314300537109375, -0.04248358681797981, -0.039242688566446304, 0.010326257906854153, -0.03199472650885582, 0.008279274217784405, -0.003955930005759001, 0.05451807752251625, -0.013624480925500393, 0.007821300067007542, 0.04170660302042961, -0.003968642093241215, 0.053862981498241425, 0.01987536996603012, -0.019811350852251053, 0.025327226147055626, -0.03696826845407486, -0.017831599339842796, -0.019258610904216766, 0.000579528568778187, -0.0024951626546680927, -0.02644186280667782, 0.041324928402900696, 0.06359915435314178, -0.006535075139254332, -0.01837475784122944, 0.04592221602797508, 0.014138688333332539, -0.0027095829136669636, 0.023073464632034302, -2.2123178496258333e-05, -0.009536691009998322, ...]",0,"[{'sent': 'its own breezy , distracted rhythms ', 'label': 1, 'sim': 0.9363362193107605}, {'sent': 'own breezy , distracted rhythms ', 'label': 1, 'sim': 0.9230661392211914}, {'sent': 'distracted rhythms ', 'label': 0, 'sim': 0.8695208430290222}, {'sent': 'undistinguished rhythm ', 'label': 0, 'sim': 0.7849590182304382}, {'sent': 'extremely languorous rhythms ', 'label': 0, 'sim': 0.7651301622390747}]",model_error,NaN


In [44]:
train_df.loc[flip_map.index].to_csv("data-flipped-on-training-side.csv", index = False)

In [45]:
train_df.loc[flip_map.index, "label"] = flip_map
print(f"✓ Relabelled {len(flip_map)} examples in the train split.")

# ------------------------------------------------------------------
# 2.  Convert back to HF Dataset objects
# ------------------------------------------------------------------
patched_train = Dataset.from_pandas(train_df, preserve_index=False)

# keep validation (and test) as-is
patched_ds = DatasetDict({
    "train"      : patched_train,
    "validation" : ds["validation"],
    # uncomment if you also want to keep test handy
    # "test"       : ds["test"],
})

# ------------------------------------------------------------------
# 3.  Save to disk in HF native format
# ------------------------------------------------------------------
save_dir = Path("./sst2_patched_arrow")
patched_ds.save_to_disk(save_dir)
print(f"✓ Patched DatasetDict saved to → {save_dir.resolve()}")

✓ Relabelled 339 examples in the train split.


Saving the dataset (0/1 shards):   0%|          | 0/67349 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/872 [00:00<?, ? examples/s]

✓ Patched DatasetDict saved to → /kaggle/working/sst2_patched_arrow
